# Minimum code for Extended Data Figure 5c qPCR panel

In [ ]:
options(warn = -1)
suppressPackageStartupMessages({
  library(tidyverse)
  library(rasilabRtemplates)
})

In [ ]:
if (!file.exists('../figures')){
    dir.create('../figures')
}

## Read in annotations

In [ ]:
annotations <- read_csv("../annotations/sampleannotations.csv") %>% 
  mutate(Well = as.character(Well),
         Plate = as.character(Plate)) %>%
  print()

## Read in Cq data from Cfx Duet

In [ ]:
cq_data <- list.files("../../../../data/qpcr/u937_gcn1_hht/", full.names = T) %>% 
   as_tibble_col("file") %>%
   mutate(data = map(file, . %>% read_csv() %>% as_tibble)) %>%
   mutate(Plate = str_extract(file, "(?<=plate)\\d")) %>%
   unnest("data") %>%
   select(Cq, Well, Plate) %>%
   rename(CT = Cq) %>%
   inner_join(annotations) %>%
   filter(!is.na(CT)) %>%
   # Remove outlier
   mutate(platewell = str_c(Plate, Well)) %>%
   filter(platewell != "7F07") %>%
   select(CT, Amplicon, sgRNA, Treatment) %>%
   mutate(Treatment = fct_relevel(Treatment, "DMSO")) %>%
   print()

## Normalize expression to GAPDH

In [ ]:
norm_ct_inverted <- cq_data  %>%
  filter(!is.na(CT)) %>%
  group_by(Amplicon, Treatment, sgRNA) %>%
  summarize(std_error = sd(CT),
            n = n(),
            CT = mean(CT)) %>%
  ungroup() %>%
  group_by(Treatment, sgRNA) %>%
  mutate(norm_ct = CT - CT[Amplicon == "GAPDH"],
         norm_error = sqrt(std_error^2 + std_error[Amplicon == "GAPDH"]^2)) %>%
  filter(Amplicon != "GAPDH") %>%
  mutate(norm_ct = 10 - norm_ct) %>%
  print()

In [ ]:
options(repr.plot.width=6, repr.plot.height=2.2)

norm_ct_inverted %>% 
  ggplot(aes(x = Treatment, y = norm_ct, ymax = norm_ct + norm_error, ymin = norm_ct - norm_error, color = sgRNA)) +
  geom_point() +
  geom_errorbar(width = 0.2) +
  facet_wrap(~Amplicon, scales = "free_y") +
  ylab("Relative mRNA level\n(-ΔΔCT)") +
  theme(axis.text.x = element_text(),
        axis.line = element_line(color = "grey")) +
  scale_color_manual(values = cbPalette) 

ggsave("../figures/egr1_jun_mrna_levels.pdf", height = 2.2, width = 6) 